In [1]:
from keras.models import model_from_json

# load json and create model
json_file = open('./model_saved/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("./model_saved/model.h5")
print("Model loaded sucessfull!")
 
model = loaded_model

Model loaded sucessfull!


In [2]:
import numpy as np
import pandas as pd
import librosa
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

max_pad_len = 174
num_rows = 40
num_columns = 174
num_channels = 1

# load features
featuresdf = pd.read_pickle('./data_processed/featuresData')

# this function load the audio and pick their features
def extract_features(file_name):
   
    try:
        
        y, sr = librosa.load(file_name, res_type='kaiser_fast') 
        
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        
        if(mfccs.shape[1]>max_pad_len):
            mfccs = mfccs[:,:max_pad_len]
        else:
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs


# function to classify the audio
def print_prediction(file_name, className):
    
    # extract audio features
    prediction_feature = extract_features(file_name) 
    
    # format input < 1, num_rows, num_columns, num_channels >
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    # predic the class
    predicted_vector = model.predict_classes(prediction_feature)
    
    # codify the label
    y = np.array(featuresdf.class_label.tolist())
    le = LabelEncoder()
    yy = to_categorical(le.fit_transform(y)) 
    
    predicted_class = le.inverse_transform(predicted_vector) 
    
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    
    predicted_proba = predicted_proba_vector[0]
    
    print("The predicted class is:", predicted_class[0], " (",format(predicted_proba[predicted_vector[0]])
          , ') - correct is: ', className, '\n') 
    
    for i in range(len(predicted_proba)): 
        
        category = le.inverse_transform(np.array([i]))
        
        if(category[0] == "disgust" or category[0] == "neutral"):
            print(category[0], " : ", format(predicted_proba[i], '.32f') )
        else:
            print(category[0], "\t : ", format(predicted_proba[i], '.32f') )
            
    print('\n\n')
    

In [3]:
from os import listdir
from os.path import isfile, join


def processDirectory(fulldatasetpath):
    
    listFear = []
    listHappy = []
    listSad = []    

    print("Classify audios from: ",fulldatasetpath)
    print("#########################################################\n\n")
    
    filenames = [f for f in listdir(fulldatasetpath) if isfile(join(fulldatasetpath, f))]
    filenames = sorted(filenames)

    # pick all test audios and put in their emotions list
    for i in range(len(filenames)):

        if(filenames[i].split('_')[2].split('.')[0]=="fear"):
            listFear.append(filenames[i])

        elif(filenames[i].split('_')[2].split('.')[0]=="happy"):
            listHappy.append(filenames[i])

        elif(filenames[i].split('_')[2].split('.')[0]=="sad"):
            listSad.append(filenames[i])



    print("Fear (from: {})".format(fulldatasetpath))
    print("---------------------------------------------------------")
    for i in range(len(listFear)):
        filename = '{}{}'.format(fulldatasetpath,listFear[i])
        print_prediction(filename,"fear")


    print("Sad (from :{})".format(fulldatasetpath))
    print("---------------------------------------------------------")
    for i in range(len(listSad)):
        filename = '{}{}'.format(fulldatasetpath,listSad[i])
        print_prediction(filename,"sad")


    print("Happy (from: {})".format(fulldatasetpath))
    print("---------------------------------------------------------")
    for i in range(len(listHappy)):
        filename = '{}{}'.format(fulldatasetpath,listHappy[i])
        print_prediction(filename,"happy")
        
        

processDirectory('./audio_valid/')
      

Classify audios from:  ./audio_valid/
#########################################################


Fear (from: ./audio_valid/)
---------------------------------------------------------
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use `model.predict()` instead.
The predicted class is: fear  ( 0.6864040493965149 ) - correct is:  fear 

fear 	 :  0.68640404939651489257812500000000
happy 	 :  0.30680778622627258300781250000000
sad 	 :  0.00678808428347110748291015625000



The predicted class is: fear  ( 0.6252041459083557 ) - correct is:  fear 

fear 	 :  0.62520414590835571289062500000000
happy 	 :  0.36977389454841613769531250000000
sad 	 :  0.005021965131163597106933593

The predicted class is: happy  ( 0.924586296081543 ) - correct is:  happy 

fear 	 :  0.04471603035926818847656250000000
happy 	 :  0.92458629608154296875000000000000
sad 	 :  0.03069762513041496276855468750000



The predicted class is: happy  ( 0.9171598553657532 ) - correct is:  happy 

fear 	 :  0.06066627800464630126953125000000
happy 	 :  0.91715985536575317382812500000000
sad 	 :  0.02217389084398746490478515625000



The predicted class is: happy  ( 0.8858270645141602 ) - correct is:  happy 

fear 	 :  0.06642685830593109130859375000000
happy 	 :  0.88582706451416015625000000000000
sad 	 :  0.04774604737758636474609375000000



The predicted class is: happy  ( 0.8323370814323425 ) - correct is:  happy 

fear 	 :  0.16577093303203582763671875000000
happy 	 :  0.83233708143234252929687500000000
sad 	 :  0.00189197459258139133453369140625



The predicted class is: happy  ( 0.8985289931297302 ) - correct is:  happy 

fear 	 :  0.05530526116490364074707031250000
happy 	 :  0.898528